# RSLC to GCOV
- This notebook converts NISAR RSLC to GCOV data by running ISCE3's `gcov.py`.
- Uses the `isce3_src` kernel (created using [Create_Environments.ipynb](https://github.com/isce-framework/sds-ondemand/blob/main/environments/Create_Environments.ipynb)).
- Can be ran locally (preferably on a GPU instance), or as a PCM job.

# Parameters
This cell is marked `parameters`, indicating the variables within can substituted when running this notebook via `papermill`.
- `data_link`: S3 url to the NISAR RSLC data to be converted into GSLC.
- `dem_s3_url`: S3 url to the DEM file to download.
- `gpu_enabled`: `1` to run using the GPU, `0` to use CPU instead. **Keep in mind that while disabling the GPU processing allows this notebook to be ran on an instance without a GPU, that does not guarantee the instance this notebook is running on is a non-GPU instance.** To run on a non-GPU instance on PCM, submit the job to a CPU-only queue.
- `s3_upload`: `1` to upload the results of this notebook to an S3 bucket, `0` to ignore this.
- `focus_config`: The runconfig passed to `focus.py`.

### Upload parameters (may be removed later in favor of automatic uploading)
- `s3_url`: S3 url to upload to results to.
- `key`: Corresponds to `aws_access_key_id` for a short-term access key stored in `~/.aws/credentials`.
- `secret`: Corresponds to `aws_secret_access_key` for a short-term access key stored in `~/.aws/credentials`.
- `token`: Corresponds to `aws_session_token` for a short-term access key stored in `~/.aws/credentials`.
- `region`: Corresponds to `region` for a short-term access key stored in `~/.aws/credentials`.

In [ ]:
data_link = 's3://nisar-st-data-ondemand/ALOS-1-data/RSLC/ALPSRP274410710-L1.0.h5' # string
dem_s3_url = 's3://nisar-st-data-ondemand/DEM-static/dem.tiff' # string
gpu_enabled = 1 # boolean
s3_upload = 1 # boolean
s3_url = 's3://nisar-st-data-ondemand/ALOS-1-data/GCOV' # string
gcov_config = '' # string
key = '' # string
secret = '' # string
token = '' # string
region = '' # string

# hysds specifications
_time_limit = 86400
_soft_time_limit = 86400
_disk_usage = '10GB'
_submission_type = 'iteration'
_label = 'RSLC to GCOV PGE'

### Pre-processing of the Parameters to convert numbers or words into `boolean` True and False values.

In [ ]:
# Convert boolean parameters because they must be specified as strings
try:
    if not isinstance(gpu_enabled, bool):
        gpu_enabled = int(gpu_enabled) > 0
except ValueError:
    if instance(gpu_enabled, str):
        gpu_enabled = gpu_enabled.lower() == 'true'
    else:
        gpu_enabled = False
print(f'{gpu_enabled=}')

try:
    if not isinstance(s3_upload, bool):
        s3_upload = int(s3_upload) > 0
except ValueError:
    if isinstance(s3_upload, str):
        s3_upload = s3_upload.lower() == 'true'
    else:
        s3_upload = False
print(f'{s3_upload=}')

# Extraneous parameters
gslc_config = '' # string
focus_config = '' # string
insar_config = '' # string

# Functions for loading runconfig files and downloading from S3 buckets

In [ ]:
import os
import yaml
import asf_search as asf
import boto3
import aws_uploader

WORKING_DIR = os.getcwd()
HOME_DIR = os.environ['HOME']
NOTEBOOK_PGE_DIR = os.environ.get('NOTEBOOK_PGE_DIR', WORKING_DIR)
ISCE3_BUILD_DIR = os.environ.get('ISCE3_BUILD_DIR', f'{HOME_DIR}/isce3/build')

DOWNLOAD_DIR = os.path.join(WORKING_DIR, 'downloads')
EXTRACT_DIR = os.path.join(WORKING_DIR, 'alos_data')
OUTPUT_DIR = os.path.join(WORKING_DIR, 'output')
RSLC_DIR = os.path.join(WORKING_DIR, 'RSLC')
GSLC_DIR = os.path.join(WORKING_DIR, 'GSLC')
PRODUCT_DIR = os.path.join(WORKING_DIR, 'product_path')

os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# RSLC
if focus_config == '':
    with open(os.path.join(NOTEBOOK_PGE_DIR, '../templates/focus.yaml'), 'r') as f:
        FOCUS_YML = yaml.safe_load(f)
else:
    print('Using custom focus.py run config...')
    FOCUS_YML = yaml.safe_load(focus_config)
    
# GSLC
if gslc_config == '':
    with open(os.path.join(NOTEBOOK_PGE_DIR, '../templates/gslc.yaml'), 'r') as f:
        GSLC_YML = yaml.safe_load(f)
else:
    print('Using custom gslc.py run config...')
    GSLC_YML = yaml.safe_load(gslc_config)
    
# GCOV
if gcov_config == '':
    with open(os.path.join(NOTEBOOK_PGE_DIR, '../templates/gcov.yaml'), 'r') as f:
        GCOV_YML = yaml.safe_load(f)
else:
    print('Using custom gcov.py run config...')
    GCOV_YML = yaml.safe_load(gcov_config)

# INSAR
if insar_config == '':
    with open(os.path.join(NOTEBOOK_PGE_DIR, '../templates/insar.yaml'), 'r') as f:
        INSAR_YML = yaml.safe_load(f)
else:
    print('Using custom insar.py run config...')
    INSAR_YML = yaml.safe_load(insar_config)

def download_alos_data(urls: list) -> list:
    """Downloads ALOS-1 data given by the asf_search URL."""
    try:
        current_dir = os.getcwd()
        user_pass_session = asf.ASFSession().auth_with_creds(username, password)
        asf.download_urls(urls=urls, path=DOWNLOAD_DIR, session=user_pass_session, processes=2)
        files = os.listdir(DOWNLOAD_DIR)
        os.chdir(EXTRACT_DIR)
        extract_dirs = []
        for f in files:
            splitext = os.path.splitext(f)
            zip_f = os.path.join(DOWNLOAD_DIR, f)
            !unzip -o {zip_f}
            extract_dirs.append(os.path.abspath(splitext[0]))
        os.chdir(current_dir)
        print('Extracted:', extract_dirs)
        return extract_dirs
    except asf.ASFAuthenticationError as e:
        print(f'Auth failed: {e}')

def download_alos_s3_data(url: str) -> str:
    """Downloads ALOS-1 data given by an S3 url."""
    try:
        current_dir = os.getcwd()
        zip_f = os.path.join(DOWNLOAD_DIR, os.path.basename(urlparse(url).path))
        if not os.path.exists(zip_f):
            print(f'Downloading {zip_f} from S3 bucket at {url}...')
            aws_uploader.AWS.download_s3(url, zip_f)
        else:
            print(f'{zip_f} already exists, skipping download...')
        os.chdir(EXTRACT_DIR)
        !unzip -o {zip_f}
        extract_dir = os.path.join(EXTRACT_DIR, os.path.basename(os.path.splitext(zip_f)[0]))
        os.chdir(current_dir)
        if os.path.isdir(extract_dir):
            print('Extracted:', extract_dir)
        else:
            raise ValueError(f'Failed to extract {extract_dir}!')
        return extract_dir
    except Exception as e:
        print(f'Exception caught while downloading ALOS-1 Data from S3: {e}')

def download_dem(url: str) -> str:
    """Downloads a DEM TIFF file given by an S3 url."""
    try:
        tiff_f = os.path.join(DOWNLOAD_DIR, os.path.basename(urlparse(url).path))
        if not os.path.exists(tiff_f):
            print(f'Downloading {tiff_f} from S3 bucket at {url}...')
            aws_uploader.AWS.download_s3(url, tiff_f)
        else:
            print(f'{tiff_f} already exists, skipping download...')

        if os.path.exists(tiff_f):
            print('Downloaded DEM:', tiff_f)
        else:
            raise ValueError(f'Failed to download {tiff_f}!')
        return tiff_f
    except Exception as e:
        print(f'Exception caught while downloading DEM Data from S3: {e}')

def write_focus_config(target_path: str, yml_path: str):
    """Writes a focus.py runconfig with the specified target path."""
    FOCUS_YML['runconfig']['groups']['worker']['gpu_enabled'] = gpu_enabled
    FOCUS_YML['runconfig']['groups']['input_file_group']['input_file_path'] = [target_path]
    FOCUS_YML['runconfig']['groups']['product_path_group']['sas_output_file'] = os.path.join(RSLC_DIR, os.path.basename(target_path))
    FOCUS_YML['runconfig']['groups']['product_path_group']['sas_config_file'] = yml_path
    with open(yml_path, 'w', encoding='utf-8') as f:
        yaml.dump(FOCUS_YML, f, default_flow_style=False)
        
def write_gslc_config(target_path: str, dem: str, yml_path: str) -> str:
    """Writes a gslc.py runconfig with the specified target path."""
    output_name = os.path.basename(os.path.splitext(target_path)[0])
    output_path = os.path.join(GSLC_DIR, f'{output_name}_GSLC.h5')
    GSLC_YML['runconfig']['groups']['worker']['gpu_enabled'] = gpu_enabled
    GSLC_YML['runconfig']['groups']['input_file_group']['input_file_path'] = target_path
    GSLC_YML['runconfig']['groups']['product_path_group']['sas_output_file'] = output_path
    # GSLC_YML['runconfig']['groups']['product_path_group']['sas_config_file'] = yml_path
    GSLC_YML['runconfig']['groups']['dynamic_ancillary_file_group']['dem_file'] = dem
    with open(yml_path, 'w', encoding='utf-8') as f:
        yaml.dump(GSLC_YML, f, default_flow_style=False)
    return output_path

def write_gcov_config(target_path: str, dem: str, yml_path: str) -> str:
    """Writes a gcov.py runconfig with the specified target path."""
    output_name = os.path.basename(os.path.splitext(target_path)[0])
    output_path = os.path.join(GSLC_DIR, f'{output_name}_GCOV.h5')
    GCOV_YML['runconfig']['groups']['worker']['gpu_enabled'] = gpu_enabled
    GCOV_YML['runconfig']['groups']['input_file_group']['input_file_path'] = target_path
    GCOV_YML['runconfig']['groups']['product_path_group']['sas_output_file'] = output_path
    GCOV_YML['runconfig']['groups']['dynamic_ancillary_file_group']['dem_file'] = dem
    with open(yml_path, 'w', encoding='utf-8') as f:
        yaml.dump(GCOV_YML, f, default_flow_style=False)
    return output_path

def write_insar_config(f1: str, f2: str, dem: str, yml_path: str) -> str:
    """Writes the INSAR runconfig with the two specified RSLCs and a DEM path."""
    product_name = INSAR_YML['runconfig']['groups']['product_path_group']['sas_output_file']
    
    INSAR_YML['runconfig']['groups']['worker']['gpu_enabled'] = gpu_enabled
    INSAR_YML['runconfig']['groups']['input_file_group']['reference_rslc_file'] = f1
    INSAR_YML['runconfig']['groups']['input_file_group']['secondary_rslc_file'] = f2
    INSAR_YML['runconfig']['groups']['dynamic_ancillary_file_group']['dem_file'] = dem
    
    b1 = os.path.basename(os.path.splitext(f1)[0])
    b2 = os.path.basename(os.path.splitext(f2)[0])
    ret = os.path.join(PRODUCT_DIR, f'{b1}_{b2}_GUNW.h5')
    INSAR_YML['runconfig']['groups']['product_path_group']['sas_output_file'] = ret

    # product_types = ['rifg', 'runw', 'gunw', 'roff', 'goff']
    # for p_type in product_types:
    #     p_type_upper = p_type.upper()
    #     INSAR_YML['runconfig']['groups']['input_file_group'][f'qa_{p_type}_input_file'] = f'{b1}_{b2}_{p_type_upper}.h5'
    
    with open(yml_path, 'w', encoding='utf-8') as f:
        yaml.dump(INSAR_YML, f, default_flow_style=False)
        
    return ret

def prepend_env_var(env_var: str, val: str) -> str:
    """Prepends a value to an environment variable without crashing."""
    if os.environ.get(env_var, '').find(val) == -1:
        os.environ[env_var] = val + ':' + os.environ.get(env_var, '')
        if os.environ[env_var].endswith(':'):
            os.environ[env_var] = os.environ[env_var][:-1]
        return os.environ[env_var]

print(WORKING_DIR)

# Run ISCE3 Python Scripts
This cell runs the python scripts:
- `gcov.py`: Converts NISAR RSLC -> GCOV

In [ ]:
from urllib.parse import urlparse

# Download the specified L0B data
target_path = os.path.join(DOWNLOAD_DIR, os.path.basename(urlparse(data_link).path))
if not os.path.exists(target_path):
    print(f'Downloading {target_path} from S3 bucket at {data_link}...')
    aws_uploader.AWS.download_s3(data_link, target_path)
else:
    print(f'{target_path} already exists, skipping download...')

# Download the DEM locally
dem_f = download_dem(dem_s3_url)

# Run gslc.py
yml_path = os.path.join(OUTPUT_DIR, 'gcov_final.yaml')
output_f = write_gcov_config(target_path, dem_f, yml_path)
print(f'Executing:\n    mamba run -n isce3_src python {ISCE3_BUILD_DIR}/packages/nisar/workflows/gcov.py {yml_path} --no-log')
!mamba run -n isce3_src python {ISCE3_BUILD_DIR}/packages/nisar/workflows/gcov.py {yml_path} --no-log

# Upload Results to S3
This cell uploads the files generated from the ISCE3 cell above to the S3 bucket defined in the `parameters` cell. It imports `aws_uploader.py`, which is a local python file.

In [ ]:
if s3_upload:
    parsed_url = urlparse(s3_url)
    bucket = parsed_url.netloc
    path = parsed_url.path
    if path.startswith('/'):
        path = path[1:]
    if not path.endswith('/'):
        path += '/'
    path = path.replace('//', '/')
    s3_object_path = f'{path}{os.path.basename(output_f)}'
    s3_object_url = f's3://{bucket}/{s3_object_path}'
    print(f'{s3_object_path=}')
    print(f'{s3_object_url=}')

    # Upload to s3 bucket
    if key != '' or secret != '' or token != '' or region != '':
        # short term access key
        uploader = aws_uploader.AWS(key, secret, token, region)
    else:
        # uses role based access
        uploader = aws_uploader.AWS('', '', '', '', configdir=False)
    uploader.upload_file(output_f, bucket, s3_object_path)
    print(f'Successfully uploaded {output_f} to {s3_object_url}')
else:
    print('Flag for manual S3 upload was not set, skipping this step...')